## Initializations

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_validate, cross_val_score, KFold, GridSearchCV, cross_val_predict
from sklearn.preprocessing import PowerTransformer
from sklearn.cross_decomposition import PLSRegression
%matplotlib inline
sns.set_style("whitegrid")
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Test datasets
solTestX = pd.read_csv("solubility/solTestX.txt",delimiter="\t") #Test set
solTestXtrans = pd.read_csv("solubility/solTestXtrans.txt",delimiter="\t")#Test set predictors after the same transformations used on the training set are applied.
solTestY = pd.read_csv("solubility/solTestY.txt",delimiter="\t") #Test set - solubility values for each compound

# Train datasets
solTrainX = pd.read_csv("solubility/solTrainX.txt",delimiter="\t") #Train set
solTrainXtrans = pd.read_csv("solubility/solTrainXtrans.txt",delimiter="\t") #Training set predictors after transformations for skewness and centering/scaling.
solTrainY = pd.read_csv("solubility/solTrainY.txt",delimiter="\t") #Train set - solubility values for each compound

solTrainXtrans.index = range(1, len(solTrainXtrans) + 1) #Get dataset indexes starting from 1
solTrainX.index = range(0, len(solTrainX)) #Get dataset indexes starting from 0
solTestX.index = range(0, len(solTestX)) #Get dataset indexes starting from 0

In [ ]:
continuous_predictors = ["MolWeight", "NumAtoms", "NumNonHAtoms", "NumBonds", "NumNonHBonds", "NumMultBonds", "NumRotBonds", "NumDblBonds", "NumAromaticBonds", "NumHydrogen", "NumCarbon", "NumNitrogen", "NumOxygen", "NumSulfer", "NumChlorine", "NumHalogen", "NumRings", "HydrophilicFactor", "SurfaceArea1", "SurfaceArea2", "x"]

## Data analysis and pre-procesing

In [ ]:
XYTrain = solTrainXtrans.join(solTrainY)

In [ ]:
#Complete dataset
dataset = pd.concat([solTrainXtrans[continuous_predictors[:-1]], solTestXtrans[continuous_predictors[:-1]]], axis=0)

In [ ]:
#Dataset transformed with Yeo-jonhson
pt = PowerTransformer()
pt.fit(dataset)
PowerTransformer(copy=True, method='yeo-johnson', standardize=True)
totalXtrans = pd.DataFrame(pt.transform(dataset), columns=continuous_predictors[:-1])

In [ ]:
dataset.skew().min()

In [ ]:
#Given transformed dataset with box-cox
dataset_analysis = {
    'Min': dataset.min(),
    'Max': dataset.max(),
    'Mean': dataset.mean(),
    'Std': dataset.std(),
    'Skewness': dataset.skew()
}
df = pd.DataFrame(dataset_analysis)
df

In [ ]:
#Dataset transformed with Yeo-jonhson

pt = PowerTransformer(method='yeo-johnson', standardize=True, copy=True).fit(dataset[continuous_predictors[:-1]])

dataset_trans = pd.DataFrame(pt.transform(dataset[continuous_predictors[:-1]]))
Xtrain = solTrainX.iloc[:,:208].join(pd.DataFrame(pt.transform(solTrainX[continuous_predictors[:-1]]), columns=continuous_predictors[:-1]))
Xtest = solTestX.iloc[:,:208].join(pd.DataFrame(pt.transform(solTestX[continuous_predictors[:-1]]), columns=continuous_predictors[:-1]))

In [ ]:
#Dataset transformed with Yeo-jonhson
dataset_analysis = {
    'Min': dataset_trans.min(),
    'Max': dataset_trans.max(),
    'Mean': dataset_trans.mean(),
    'Std': dataset_trans.std(),
    'Skewness': dataset_trans.skew()
}

df2 = pd.DataFrame(dataset_analysis)
df2

In [ ]:
for x in continuous_predictors[:-1]:
    sns.lmplot(x=x, y="x", data=XYTrain)    
plt.show()

In [ ]:
solTrainXcontinous = solTrainXtrans.iloc[:,208:]

In [ ]:
corr = XYTrain[continuous_predictors[:-1]].corr()
mask = np.zeros_like(corr)

f, ax = plt.subplots(figsize=(18, 12))

sns.heatmap(
    corr,
    annot=True,
    fmt=".2f",
    linewidths=.5,
    ax=ax, 
    vmin=-1, vmax=1, 
    cmap="viridis",
    cbar_kws={"shrink": .7}
)

In [ ]:
corr = XYTrain[continuous_predictors].corr()
mask = np.zeros_like(corr)

f, ax = plt.subplots(figsize=(18, 12))

sns.heatmap(
    corr,
    annot=True,
    fmt=".2f",
    linewidths=.5,
    ax=ax, 
    vmin=-1, vmax=1, 
    cmap="viridis",
    cbar_kws={"shrink": .7}
)

## Oridnary linear regression

In [ ]:
lm = LinearRegression()
model = lm.fit(solTrainXtrans, solTrainY)
Ypred = model.predict(solTestXtrans)

print('RMSE', np.sqrt(mean_squared_error(solTestY, Ypred)))
print('R2', r2_score(solTestY, Ypred))

## Cross-validation

In [ ]:
lr = LinearRegression()
MSEs = cross_val_score(lm, solTrainXtrans, solTrainY, scoring='neg_mean_squared_error', cv=10)
mean_MSE = np.mean(np.abs(MSEs))
print(np.sqrt(mean_MSE))

In [ ]:
kf = KFold(n_splits=10, random_state=100, shuffle=True)

R2 = []
RMSE = []

for train_index, test_index in kf.split(solTrainXtrans):
    x_train = solTrainXtrans.iloc[train_index]
    y_train = solTrainY.iloc[train_index]
    
    x_test = solTrainXtrans.iloc[test_index]
    y_test = solTrainY.iloc[test_index]
    
    lm = LinearRegression()
    lm.fit(x_train, y_train)
    
    preds = lm.predict(x_test)
    
    RMSE.append(np.sqrt(mean_squared_error(y_test, preds)))
    R2.append(r2_score(y_test, preds))

print('RMSE', np.mean(RMSE))
print('R2', np.mean(R2))

# OLR Plots

In [ ]:
Ypred_ = sum(Ypred.tolist(), [])
solTestY_ = sum(solTestY.values.tolist(), [])
resid = [y_test - y_pred for y_test, y_pred in zip(solTestY_, Ypred_)]

df1 = pd.DataFrame({'Prediction':Ypred_, 'Observation':solTestY_})
sns.lmplot(x='Prediction', y="Observation", data=df1, fit_reg=False)

df2 = pd.DataFrame({'Prediction': Ypred_, 'Residue': resid})
sns.lmplot(x='Prediction', y="Residue", data=df2, fit_reg=False)


## Removing high correlated predictors

In [ ]:
corr_matrix = solTrainX.corr().abs()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

newX = solTrainX.drop(to_drop, axis=1)
newXtest = solTestX.drop(to_drop, axis=1)
print(to_drop)

model2 = LinearRegression().fit(newX, solTrainY)
Ypred2 = model2.predict(newXtest)

print('RMSE', np.sqrt(mean_squared_error(solTestY, Ypred2)))
print('R2', r2_score(solTestY, Ypred2))

# Penalized linear regression model L2 (Ridge)

In [ ]:
lr = LinearRegression()
lr.fit(solTrainXtrans, solTrainY)

rr = Ridge(alpha=0.03)
rr.fit(solTrainXtrans, solTrainY)

rr3 = Ridge(alpha=100)
rr3.fit(solTrainXtrans, solTrainY)

plt.plot(range(209, 229), rr.coef_[0][208:],alpha=0.7,linestyle='none',marker='X',markersize=9,color='red',label=r'Ridge; $\alpha = 0.01$',zorder=7) # zorder for ordering the markers
plt.plot(range(209, 229), rr3.coef_[0][208:],alpha=0.5,linestyle='none',marker='s',markersize=9,color='blue',label=r'Ridge; $\alpha = 100$') # alpha here is for transparency
plt.plot(range(209, 229), lr.coef_[0][208:],alpha=0.4,linestyle='none',marker='o',markersize=9,color='green',label='Linear Regression')

plt.xlabel('Índice Coeficiente',fontsize=14)
plt.ylabel('Magnitude do coeficiente',fontsize=14)
plt.legend(fontsize=12,loc=1)
plt.show()

In [ ]:
ridge = Ridge()
params = {'alpha': np.linspace(0.1, 15,100)}
ridge_regressor = GridSearchCV(ridge, params, scoring=('r2', 'neg_mean_squared_error'), cv=10, refit='neg_mean_squared_error')
ridge_regressor.fit(solTrainXtrans, solTrainY)

print(ridge_regressor.best_params_)
print(np.sqrt(np.abs(ridge_regressor.best_score_)))

In [ ]:
alphas = np.linspace(1,20,30)
cv_ridge = [np.sqrt(-cross_val_score(Ridge(alpha = alpha), solTrainXtrans, solTrainY, scoring="neg_mean_squared_error", cv = 10)).mean() for alpha in alphas]
cv_ridge = pd.Series(cv_ridge, index = alphas)
cv_ridge.plot(title = "Validation")

alpha_min = alphas[cv_ridge.tolist().index(cv_ridge.min())]

plt.xlabel("alpha")
plt.ylabel("RMSE")
plt.plot(alpha_min, cv_ridge.min(), marker='o', markersize=9, color='red')
plt.show()
print(alpha_min)

In [ ]:
alphas = np.linspace(1,20,30)
R2 = []
RMSE = []

for alpha in alphas:
    rr = RidgeCV(alphas=[alpha], cv=10) 
    rmodel = rr.fit(solTrainXtrans, solTrainY)
    Ypred = rmodel.predict(solTestXtrans)
    
    R2.append(r2_score(solTestY, Ypred))
    RMSE.append(np.sqrt(mean_squared_error(solTestY, Ypred)))

df = pd.DataFrame(
    {
        'alpha': alphas,
        'RMSE': RMSE,
        'R2': R2
    }
)
df

## Parcial Least Squares regression (PLS)

In [ ]:
components = range(1,30)

cv_pls = [np.sqrt(-cross_val_score(PLSRegression(n_components=comps), solTrainXtrans, solTrainY, scoring='neg_mean_squared_error', cv=10)).mean() for comps in components]
cv_pls = pd.Series(cv_pls, index = components)

cv_pls.plot(title='Validation')

n_comp_min = components[cv_pls.tolist().index(cv_pls.min())]

print(n_comp_min, cv_pls.min())

plt.xlabel("components")
plt.ylabel("RMSE")
plt.plot(n_comp_min, cv_pls.min(), marker='o', markersize=9, color='red')
plt.show()

In [ ]:
pls = PLSRegression(n_components = 19)

pls.fit(solTrainXtrans, solTrainY)
Ypred_pls = pls.predict(solTestXtrans)
Ytest_pls = solTestY

score = r2_score(Ytest_pls, Ypred_pls)
rmse = np.sqrt(mean_squared_error(Ytest_pls, Ypred_pls))

print('R2', score)
print('RMSE', rmse)

In [ ]:
Ypred_ = sum(Ypred_pls.tolist(), [])
Ytest_ = sum(Ytest_pls.values.tolist(), [])
resid = [y_test - ypred for y_test, ypred in zip(Ytest_, Ypred_)]

df1 = pd.DataFrame({'Prediction': Ypred_, 'Observation': Ytest_})
sns.lmplot(x='Prediction', y="Observation", data=df1, fit_reg=False)

df2 = pd.DataFrame({'Prediction': Ypred_, 'Residue': resid})
sns.lmplot(x='Prediction', y="Residue", data=df2, fit_reg=False)

plt.show()